In [1]:
from room import Room
from person import Person
from action import Action
import metacode_helpers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# initialise all actions
ALL_POSSIBLE_ACTIONS = metacode_helpers.initialise_all_actions()
ALL_CONVERSATION_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "conversation"]
ALL_ROOM_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "room"] 

In [4]:
# Create some rooms
HALL = Room("Hall")
KITCHEN = Room("Kitchen")
LOUNGE = Room("Lounge")

# Add adjacencies to the rooms (HALL is adjacent to KITCHEN and LOUNGE)
HALL.add_adjacent_room(KITCHEN)
HALL.add_adjacent_room(LOUNGE)

# Assign the Hall to be the entry to the house
HOUSE_ENTRY_ROOM = HALL 

ROOMS = [HALL, KITCHEN, LOUNGE]

In [5]:
ALL_POSSIBLE_ACTIONS

array([shouts, shares, leaves_conversation, makes_smalltalk,
       makes_deeptalk, gives_controversial_opinion, changes_the_topic,
       shows_interest, shows_concern, laughs, jokes, gives_an_insult,
       brags, yawns, supports, rants, stands_in_corner, leaves_room,
       starts_conversation], dtype=object)

In [6]:
from collections import defaultdict

num_runs, num_rounds, num_people = 10, 10, 32


action_log = defaultdict(list)
person_before_action_log = defaultdict(list)
person_after_action_log = defaultdict(list)
partner_before_action_log = defaultdict(list)
partner_after_action_log = defaultdict(list)

def log_snapshot(log, snapshot, run_number, round_number, turn_number, prefix=""):
    log["run_number"].append(run_number)
    log["round_number"].append(round_number)
    log["turn_number"].append(turn_number)
    for key, val in snapshot.items():
        log[prefix + "_" + key].append(val)

for run_number in range(num_runs):
    all_people = metacode_helpers.initialise_all_people(all_possible_actions = ALL_POSSIBLE_ACTIONS, num_people=num_people, rooms=ROOMS)
    turn_number = 0
    for round_number in range(num_rounds):
        for person in all_people:
            partner = person.get_conversation_partner()
            log_snapshot(person_before_action_log, person.get_snapshot(), run_number, round_number, turn_number, "person_before_action")
            if person.has_conversation_partner():
                log_snapshot(partner_before_action_log, person.get_conversation_partner().get_snapshot(), run_number, round_number, turn_number, "partner_before_action")

            # The person can start a conversation only if there's someone free to chat in their current room.
            available_room_act = ALL_ROOM_ACTIONS if person.get_location_state().is_someone_free_to_chat(person) else [action for action in ALL_ROOM_ACTIONS if action.get_name() != "starts_conversation"]
            #print(available_room_act, "free?", person.get_location_state().is_someone_free_to_chat(person))
            # If in a conversation, the person can select any action, including leaving.
            action = person.action_selection(available_conv_act=ALL_CONVERSATION_ACTIONS) if person.has_conversation_partner() else person.action_selection(available_room_act=available_room_act)
            
            # If in a conversation, update the person & conversation partner's emotional state according to the action
            if person.has_conversation_partner():
                # Change this person's emotional state 
                person.update_emotional_state_vector(action, isOwnAction = True)
                # Change the partner's emotional state 
                partner.update_emotional_state_vector(action, isOwnAction = False)

                if action.get_name() == "leaves_conversation":
                    # The conversation has ended, so both people now don't have conversation partners.
                    person.set_conversation_partner(None)
                    partner.set_conversation_partner(None)
                    
                    
            else: # Otherwise, deal with the room action
                if action.get_name() == "leaves_room":
                    next_room = person.get_next_room()
                    person.move_to_room(next_room)          
                if action.get_name() == "starts_conversation":
                    # NOTE whether this is possible is handled by the action selection function
                    # Find a person in the room who is free to chat
                    current_room = person.get_location_state()
                    partner = current_room.get_new_conversation_partner_for(person)
                    person.set_conversation_partner(partner)
                    partner.set_conversation_partner(person)
            
            log_snapshot(action_log, action.get_snapshot(), run_number, round_number, turn_number)    
            log_snapshot(person_after_action_log, person.get_snapshot(), run_number, round_number, turn_number, "person_after_action")
            if partner:
                log_snapshot(partner_after_action_log, partner.get_snapshot(), run_number, round_number, turn_number, "partner_after_action")

            message = person.get_name(), ' '.join(action.get_name().split("_"))
            #print(round_number)
            #print(turn_number)
            #if partner:
                #print(message, 'To:', partner.get_name())
            #else:
                #print(message)
            turn_number += 1

In [7]:
action_log_df = pd.DataFrame(action_log)
person_before_action_log_df = pd.DataFrame(person_before_action_log)
person_after_action_log_df = pd.DataFrame(person_after_action_log)
partner_before_action_log_df = pd.DataFrame(partner_before_action_log)
partner_after_action_log_df = pd.DataFrame(partner_after_action_log)


#Write to CSV files
action_log_df.to_csv('logs/action.csv')
person_before_action_log_df.to_csv('logs/personbefore.csv')
person_after_action_log_df.to_csv('logs/personafter.csv')
partner_before_action_log_df.to_csv('logs/partnerbefore.csv')
partner_after_action_log_df.to_csv('logs/partnerafter.csv')

In [8]:
all = pd.merge(action_log_df, person_before_action_log_df, on=['run_number', 'round_number', 'turn_number'], how="outer")
all = pd.merge(all, person_after_action_log_df, on=['run_number', 'round_number', 'turn_number'], how="outer")
all = pd.merge(all, partner_before_action_log_df, on=['run_number', 'round_number', 'turn_number'], how="outer")
all = pd.merge(all, partner_after_action_log_df, on=['run_number', 'round_number', 'turn_number'], how="outer")
all.to_csv("logs/all_together.csv")